#### DNN기반 회귀 모델 구현
 - 데이터셋 : iris.csv
 - feature : 3개 Sepal_Length, Sepal_Width, Petal_Length
 - target/label : 1개 Petal_Width
 - 학습방법 : 지도학습 > 회귀
 - 알고리즘 : 인공신경망(ANN) -> MLP, DNN : 은닉층이 많은 구성
 - 프레임워크 : Pytorch

[1] 모듈 로딩 및 데이터 준비
***

In [2]:
# - 모델 관련
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torchmetrics.regression import R2Score, MeanSquaredLogError
from torchinfo import summary

# - 데이터 관련
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split

In [3]:
# 활용 패키지 버전 체크  ==> 사용자 정의 함수로 구현해놓기
def checkversion() :
    print(f'Pytorch v {torch.__version__}')
    print(f'pandas v {pd.__version__}')

In [4]:
#### 데이터 로딩
irisDF = pd.read_csv(r'C:\Users\hoon\Desktop\경대 KDT 6기\EX_PANDAS6-main\EX_PANDAS6-main\TORCH_DL\data\iris.csv',usecols=[0,1,2,3])
irisDF.head(2)

,sepal.length,sepal.width,petal.length,petal.width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2


[2] 모델 클래스 설계 및 정의 <hr>
 - 클래스 목적 : iris 데이터 학습 및 추론
 - 클래스 이름 : IrisRegModel
 - 부모 클래스 : nn.Module
 - 매개변수    : 층별 입/출력 개수 고정 => 매개변수 필요 없음 
 - 속성/필드   :  
 - 클래스 기능 : _ _ init _ _() : 모델 구조 설정, forward() : 순방향 학습  <= 오버라이딩(overriding)
 - 클래스 구조 
   * 입력층 : 입력 3개     출력 10개
   * 은닉층 : 입력 10개    출력 30개
   * 출력층 : 입력 30개     출력 1개

 - 활성화함수
   * 클래스 형태 ==> nn.MELoss, nn.ReLU  -> _ _ init _ _() 메서드에서 사용
   * 함수 형태  ==> torch.nn.functional 아래에 -> forward() 메서드에서 사용

In [5]:
class irisRegModel(nn.Module) :

    # 모델 구조 구성 및 인스턴스 생성 메서드
    def __init__(self) :
        super().__init__()
        self.in_layer = nn.Linear(3,10)
        self.h_layer = nn.Linear(10,30)
        self.out_layer = nn.Linear(30,1)

    # 순방향 학습 진행 메서드
    def forward(self, data) :
        # 입력층
        y = self.in_layer(data)             # f1w1+f2w2+f3w3+...+f10w10+b
        F.relu(y)                           # relu => y 값의 범위 : 0 < y
        
        # 은닉층 : 10개의 숫자값이 들어옴(>=0)
        y = self.h_layer(y)
        F.relu(y)

        # 출력층 : 30개의 숫자값이 들어옴(>=0)
        return self.out_layer(y)
        

In [6]:
model = irisRegModel()

In [7]:
summary(model, input_size=(100000,3))

Layer (type:depth-idx)                   Output Shape              Param #
irisRegModel                             [100000, 1]               --
├─Linear: 1-1                            [100000, 10]              40
├─Linear: 1-2                            [100000, 30]              330
├─Linear: 1-3                            [100000, 1]               31
Total params: 401
Trainable params: 401
Non-trainable params: 0
Total mult-adds (M): 40.10
Input size (MB): 1.20
Forward/backward pass size (MB): 32.80
Params size (MB): 0.00
Estimated Total Size (MB): 34.00

[3] 데이터셋 클래스 설계 및 정의 <hr>
 - 데이터셋 : iris.csv
 - 피쳐개수 : 3개
 - 타겟개수 : 1개
 - 클래스명 :  IrisDataset
 - 부모클래스 : utils.data.Dataset
 - 속성/필드  : featureDF, targetDF, n_rows, n_features
 - 필수메서드 : 
    *  _ _ init _ _(self) : 데이터셋 저장 및 전처리, 기타 필요한 추가 속성 설정
    * _ _ len _ _(self) : 데이터의 개수 반환
    * _ _ getitem _ _(self, index) : 특정 인덱스의 피쳐와 타겟 반환

In [8]:
class IrisDataset(Dataset) :

    def __init__(self,featureDF, targetDF) :
        self.featureDF = featureDF
        self.targetDF = targetDF
        self.n_rows = featureDF.shape[0]
        self.n_features = featureDF.shape[1]

    def __len__(self) :
        return self.n_rows

    def __getitem__(self, index) :
        
        # 텐서화
        featureTS = torch.FloatTensor(self.featureDF.iloc[index].values)
        targetTS = torch.FloatTensor(self.targetDF.iloc[index].values)

        # 피쳐와 타겟 반환
        return featureTS, targetTS

In [15]:
## 데이터셋 인스턴스 생성

# - DataFrame에서 피쳐와 타겟 추출
featureDF = irisDF[irisDF.columns[:-1]]
targetDF = irisDF[irisDF.columns[-1:]]

print(targetDF.shape)

# - 커스텀 데이터셋 인스턴스 생성
irisDS = IrisDataset(featureDF,targetDF)
irisDS.targetDF

(150, 1)


,petal.width
0,0.2
1,0.2
2,0.2
3,0.2
4,0.2
...,...
145,2.3
146,1.9
147,2.0
148,2.3


[4] 학습 준비
 - 학습횟수 : EPOCH          <- 처음 ~ 끝까지 공부하는 단위  
 - 배치크기 : BATCH_SIZE     <- 한번에 학습할 데이터 양
 - 위치지정 : DEVICE         <- 텐서 저장 및 실행 위치 (GPU/CPU)
 - 학습률(lr) : 가중치와 절편 업데이트 시 경사하강법으로 업데이트 간격 설정 (0.001 ~ 0.1) <- 하이퍼파라미터

In [10]:
### 학습 진행 관련 설정값
EPOCH = 1
BATCH_SIZE = 10
BATCH_CNT = irisDF.shape[0]//BATCH_SIZE
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LR = 0.001

 - 학습 준비에 필요한 것
    * 인스턴스/객체 : 모델, 데이터셋, 최적화, (, 손실함수, 성능지표)

In [11]:
# 모델 인스턴스
model = irisRegModel()

# 데이터셋 인스턴스
X_train, X_test, y_train, y_test = train_test_split(featureDF,targetDF,random_state = 1)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,random_state = 1)

trainDS = IrisDataset(X_train,y_train)
valDS = IrisDataset(X_val,y_val)
testDS = IrisDataset(X_test, y_test)

# 데이터로더 인스턴스
trainDL = DataLoader(trainDS,batch_size=BATCH_SIZE)


In [12]:
## [테스트] 데이터 로더
for feature, target in trainDL :
    print(feature.shape, target.shape)
    break

torch.Size([10, 3]) torch.Size([10, 1])


In [13]:
# 최적화 인스턴스 - 최소의 손실을 만드는 w,b 찾는 작업 해주는 인스턴스
# 최적화 인스턴스에 model.parameters() 전달 필요
optimizer = optim.Adam(model.parameters(),lr=LR)


# 손실함수 인스턴스 > 회귀 - MSE, MAE, RMSE, ...
regLoss = nn.MSELoss()

[5] 학습 진행<hr>

In [14]:
## 학습의 효과 확인을 위해 손실값과 성능평가값 저장 필요
LOSS_HISTORY, SCORE_HISTORY = [[],[]],[[],[]]

for epoch in range(EPOCH) :
    # 학습 모드로 모델 설정
    model.train()


    # 배치크기만큼 데이터 로딩해서 학습 진행
    loss_total , score_total = 0, 0
    for featureTS, targetTS in trainDL :
        
        # 학습 진행
        pre_y = model(featureTS)

        # 손실 계산
        loss = regLoss(pre_y,targetTS)
        loss_total += loss.item()

        # 성능 평가
        score = R2Score()(pre_y,targetTS)
        score_total += score.item()

        # 최적화 진행
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # 에포크 당 검증기능
    # 모델 검증 모드 설정
    model.eval()
    with torch.no_grad():
        # 검증 데이터셋
        val_featureTS = torch.FloatTensor(valDS.featureDF.values)
        val_targetTS = torch.FloatTensor(valDS.targetDF.values)
        # 추론/평가
        pre_val = model(val_featureTS)
        # 손실계산
        loss_val = regLoss(pre_val, val_targetTS)
        # 성능평가
        score_val = R2Score()(pre_val, val_targetTS)



    # 손실값과 성능평가값 저장
    LOSS_HISTORY[0].append(loss_total/BATCH_CNT) 
    SCORE_HISTORY[0].append(score_total/BATCH_CNT)
    LOSS_HISTORY[1].append(loss_val) 
    SCORE_HISTORY[1].append(score_val)  